In [1]:
import pyspark
from pyspark.sql import functions as sf
import requests
import json
import pathlib as path
import os
from dotenv import load_dotenv

In [2]:
# load config.env variables
env_path = path.Path('.') / 'config.env'
load_dotenv(env_path)
api_key = os.getenv('api_key')
print(len(api_key))

64


In [14]:
# Open openaq API and get a sample of measurements
headers = {"X-API-Key": api_key}
response = requests.get(
    "https://api.openaq.org/v3/locations",
    params={"limit": 50},
    headers=headers,
)

print(response.status_code)
print(response.ok)

data = response.json()
print(json.dumps(data, indent=2))


200
True
{
  "meta": {
    "name": "openaq-api",
    "website": "/",
    "page": 1,
    "limit": 50,
    "found": ">50"
  },
  "results": [
    {
      "id": 3,
      "name": "NMA - Nima",
      "locality": null,
      "timezone": "Africa/Accra",
      "country": {
        "id": 152,
        "code": "GH",
        "name": "Ghana"
      },
      "owner": {
        "id": 4,
        "name": "Unknown Governmental Organization"
      },
      "provider": {
        "id": 209,
        "name": "Dr. Raphael E. Arku and Colleagues"
      },
      "isMobile": false,
      "isMonitor": true,
      "instruments": [
        {
          "id": 2,
          "name": "Government Monitor"
        }
      ],
      "sensors": [
        {
          "id": 6,
          "name": "pm10 \u00b5g/m\u00b3",
          "parameter": {
            "id": 1,
            "name": "pm10",
            "units": "\u00b5g/m\u00b3",
            "displayName": "PM10"
          }
        },
        {
          "id": 5,
          "nam

In [15]:
# Check meta to see total records
data = response.json()
print("Meta info:", json.dumps(data['meta'], indent=2))

Meta info: {
  "name": "openaq-api",
  "website": "/",
  "page": 1,
  "limit": 50,
  "found": ">50"
}


In [ ]:
import time

headers = {"X-API-Key": api_key}
all_locations = []
page = 1

while True:
    response = requests.get(
        "https://api.openaq.org/v3/locations",
        params={"limit": 100, "page": page},  # 100 per page
        headers=headers
    )
    data = response.json()
    all_locations.extend(data['results'])
    
    print(f"Page {page}: {len(all_locations)} total so far")
    
    if len(data['results']) < 100:  # last page
        break
    
    page += 1
    time.sleep(1.1)  # 60 requests/min = ~1 sec between requests

print(f"Done! Total locations: {len(all_locations)}")

In [18]:
# STEP 2: Check measurements data size for a specific location
headers = {"X-API-Key": api_key}

# Use location ID 8118 (New Delhi) that we found earlier
location_id = 8118

response = requests.get(
    f"https://api.openaq.org/v3/locations/{location_id}/measurements",
    params={"limit": 100, "page": 1},
    headers=headers
)

if response.status_code == 200:
    data = response.json()
    print("Meta:", json.dumps(data.get('meta'), indent=2))
    
    # Estimate size
    import sys
    record_size = sys.getsizeof(str(data['results'][0]))
    print(f"\n~{record_size} bytes per measurement")
    print(f"Sample measurement:", json.dumps(data['results'][0], indent=2))
    
    # Rough calculation
    print(f"\nIf 1 million measurements: ~{(record_size * 1000000) / 1024 / 1024:.2f} MB")
else:
    print(f"Error {response.status_code}: {response.text}")

Error 404: {"detail":"Not Found"}


In [19]:
# Check if location metadata has a link to measurements
headers = {"X-API-Key": api_key}
response = requests.get(
    "https://api.openaq.org/v3/locations/8118",
    headers=headers
)

if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2))
    print("\n=== Check for measurements link or sensor info ===")
else:
    print(f"Error {response.status_code}: {response.text}")

{
  "meta": {
    "name": "openaq-api",
    "website": "/",
    "page": 1,
    "limit": 100,
    "found": 1
  },
  "results": [
    {
      "id": 8118,
      "name": "New Delhi",
      "locality": "India",
      "timezone": "Asia/Kolkata",
      "country": {
        "id": 9,
        "code": "IN",
        "name": "India"
      },
      "owner": {
        "id": 4,
        "name": "Unknown Governmental Organization"
      },
      "provider": {
        "id": 119,
        "name": "AirNow"
      },
      "isMobile": false,
      "isMonitor": true,
      "instruments": [
        {
          "id": 2,
          "name": "Government Monitor"
        }
      ],
      "sensors": [
        {
          "id": 23534,
          "name": "pm25 \u00b5g/m\u00b3",
          "parameter": {
            "id": 2,
            "name": "pm25",
            "units": "\u00b5g/m\u00b3",
            "displayName": "PM2.5"
          }
        }
      ],
      "coordinates": {
        "latitude": 28.63576,
        "longi

In [20]:
# Calculate realistic data scope
headers = {"X-API-Key": api_key}

# 1. Get total locations (already know ~10k)
# 2. Check one sensor's measurement count
response = requests.get(
    "https://api.openaq.org/v3/sensors/23534/measurements",
    params={"limit": 1, "page": 1},
    headers=headers
)

if response.status_code == 200:
    data = response.json()
    print("Meta:", json.dumps(data.get('meta'), indent=2))
    print("\nDate range:", data['results'][0] if data['results'] else "No data")
    
    # Estimate: if sensor 23534 has data from 2016-2025 (9 years)
    # Hourly = 9 * 365 * 24 = ~78,840 measurements per sensor
    # 10k locations * 1-5 sensors * 78k measurements = ???
else:
    print(f"Error: {response.text}")

Meta: {
  "name": "openaq-api",
  "website": "/",
  "page": 1,
  "limit": 1,
  "found": ">1"
}

Date range: {'value': 270.0, 'flagInfo': {'hasFlags': False}, 'parameter': {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': None}, 'period': {'label': 'raw', 'interval': '01:00:00', 'datetimeFrom': {'utc': '2016-11-09T17:30:00Z', 'local': '2016-11-09T23:00:00+05:30'}, 'datetimeTo': {'utc': '2016-11-09T18:30:00Z', 'local': '2016-11-10T00:00:00+05:30'}}, 'coordinates': None, 'summary': None, 'coverage': {'expectedCount': 1, 'expectedInterval': '01:00:00', 'observedCount': 1, 'observedInterval': '01:00:00', 'percentComplete': 100.0, 'percentCoverage': 100.0, 'datetimeFrom': {'utc': '2016-11-09T17:30:00Z', 'local': '2016-11-09T23:00:00+05:30'}, 'datetimeTo': {'utc': '2016-11-09T18:30:00Z', 'local': '2016-11-10T00:00:00+05:30'}}}
